<a href="https://colab.research.google.com/github/CSID-DGU/2021-2-CSC4031-Meeting/blob/main/%EC%9A%94%EC%95%BD_%ED%85%8C%EC%8A%A4%ED%8A%B8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#구글계정 마운트
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip3 install torch==1.8.1+cpu torchvision==0.9.1+cpu torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 169.1 MB 76 kB/s 
     |████████████████████████████████| 13.3 MB 41.9 MB/s 
     |████████████████████████████████| 1.9 MB 31.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1+cpu which is incompatible.


In [3]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

     |████████████████████████████████| 60 kB 8.2 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191620 sha256=4ca54b5d5ae253356a9b84bc2d879fb5269ede47de8af29bd805f1288630c309
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
     | 202 kB 7.3 MB/s
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 
Cloning into 'pyrouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 14.94 MiB/s, done.
Resolving deltas: 100% (109/109), done.
2021-11-01 10:30:47,053 [MainThread  ] [INFO ]  Set ROUGE h

In [4]:

!pip install transformers
!pip install tensorboardX
!pip install easydict

     |████████████████████████████████| 3.1 MB 28.2 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 38.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
     |████████████████████████████████| 596 kB 28.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 124 kB 37.6 MB/s 


In [5]:
%cd /content
!git clone https://github.com/HaloKim/KorBertSum.git
%cd /content/KorBertSum

/content
Cloning into 'KorBertSum'...
remote: Enumerating objects: 11142, done.
remote: Counting objects: 100% (10840/10840), done.
remote: Compressing objects: 100% (7610/7610), done.
remote: Total 11142 (delta 3259), reused 10780 (delta 3220), pack-reused 302
Receiving objects: 100% (11142/11142), 18.97 MiB | 16.95 MiB/s, done.
Resolving deltas: 100% (3434/3434), done.
/content/KorBertSum


In [6]:
import os

os.chdir('/content/KorBertSum/src')

In [7]:
import torch
import numpy as np
from models import data_loader, model_builder
from models.model_builder import Summarizer
from others.logging import logger, init_logger
from models.data_loader import load_dataset
from transformers import BertConfig, BertTokenizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
import easydict

In [8]:
def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        logger.info('* number of parameters: %d' % n_params)

    return trainer


class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summ(self, test_iter, step, cal_lead=False, cal_oracle=False):
      """ Validate model.
          valid_iter: validate data iterator
      Returns:
          :obj:`nmt.Statistics`: validation loss statistics
      """
      # Set model in validating mode.
      def _get_ngrams(n, text):
          ngram_set = set()
          text_length = len(text)
          max_index_ngram_start = text_length - n
          for i in range(max_index_ngram_start + 1):
              ngram_set.add(tuple(text[i:i + n]))
          return ngram_set

      def _block_tri(c, p):
          tri_c = _get_ngrams(3, c.split())
          for s in p:
              tri_s = _get_ngrams(3, s.split())
              if len(tri_c.intersection(tri_s))>0:
                  return True
          return False

      if (not cal_lead and not cal_oracle):
          self.model.eval()
      stats = Statistics()

      with torch.no_grad():
          for batch in test_iter:
              src = batch.src
              labels = batch.labels
              segs = batch.segs
              clss = batch.clss
              mask = batch.mask
              mask_cls = batch.mask_cls

              if (cal_lead):
                  selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
              elif (cal_oracle):
                  selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                  range(batch.batch_size)]
              else:
                  sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)
                  sent_scores = sent_scores + mask.float()
                  sent_scores = sent_scores.cpu().data.numpy()
                  selected_ids = np.argsort(-sent_scores, 1)
      return selected_ids



    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

class BertData():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]
        idxs = [i for i, s in enumerate(src) if (len(s) > 1)]

        src = [src[i][:2000] for i in idxs]
        src = src[:1000]

        if (len(src) < 3):
            return None

        src_txt = [' '.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        src_txt = [original_src_txt[i] for i in idxs]
        tgt_txt = None
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

def _lazy_dataset_loader(pt_file):
  yield  pt_file

In [15]:
args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'test',
    "bert_data_path":'/content/KorBertSum/bert_data/korean',
    "model_path":'./models/bert_classifier',
    "result_path":'./results',
    "temp_dir":'./temp',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'/content/KorBertSum/logs/log.log',
    "test_from":'/content/gdrive/MyDrive/<융캡> 회의중 드라이브/문서요약/model_step_24000.pt의 사본'
})
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

In [16]:
def test(args, input_list, device_id, pt, step):
  init_logger(args.log_file)
  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  device_id = 0 if device == "cuda" else -1

  cp = args.test_from
  try:
    step = int(cp.split('.')[-2].split('_')[-1])
  except:
    step = 0

  device = "cpu" if args.visible_gpus == '-1' else "cuda"
  if (pt != ''):
      test_from = pt
  else:
      test_from = args.test_from
  logger.info('Loading checkpoint from %s' % test_from)
  checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
  opt = vars(checkpoint['opt'])
  for k in opt.keys():
      if (k in model_flags):
        setattr(args, k, opt[k])

  config = BertConfig.from_pretrained('bert-base-multilingual-cased')
  model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
  model.load_cp(checkpoint)
  model.eval()

  test_iter = data_loader.Dataloader(args, _lazy_dataset_loader(input_list),
                                args.batch_size, device,
                                shuffle=False, is_test=True)
  trainer = build_trainer(args, device_id, model, None)
  result = trainer.summ(test_iter,step)
  return result, input_list

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

In [17]:
def txt2input(text):
  data = list(filter(None, text.split('\n')))
  bertdata = BertData()
  txt_data = bertdata.preprocess(data)
  data_dict = {"src":txt_data[0],
               "labels":[0,1,2],
               "segs":txt_data[2],
               "clss":txt_data[3],
               "src_txt":txt_data[4],
               "tgt_txt":None}
  input_data = []
  input_data.append(data_dict)
  return input_data

In [18]:
text = '''
임진년의 외인들이 난을 버렸다 그래서 임진왜란 인데요.
자 지도 한번 보실께요 자 중국이 있고 우리나라가 있고 이렇게 일본이 펼쳐집니다.
여기는 일본을 주목 하셔야 되는데요.
백년동안 자기들끼리 치고받고 있어요 이걸 전국시대 라고 합니다.
당시 도요토미 히데요시가 조총으로 통일합니다.
통일을 했는데 문제는 너무 불안정한 통일 이였어요.
내부적인 결속이 필요했습니다.
자 또한 뭐냐 자 무사들은 어떡할 거에요.
무사 라는 건 전쟁통에 능력을 발휘하는 사람들인데 이 평화의 시기에  뭐 하는 일 없이 백수로 죽으란 얘기 아닙니까 
안되겠다 이들의 에너지를 표출할 수 있는 분출구를 찾아야겠다는 생각을 합니다.
자 이런 상황에서 일본이 드디어 우리나라를 침공을 한 겁니다
자 부산으로 바로 쳐 들어 온거죠
아 그들의 생각은 뭐냐 조선에서 힘 빼겠다고 그런거 필요없어요
속도전으로 재빠르게 서울까지 치고 올라가서 임금을 인질로 잡는 겁니다
그래서 그를 볼모 삼아 서 군사들을 모아 그리고 불량배를 모아 그래가지고 총알받이로 조선인들의 앞장 세우고
이제 중국도 쳐들어 가려고 했던 거죠 그런데 세상에 우리 선조가 상상도 하지 못한 업적을 남기시죠
한양을 버리고 의주로 도망을 갑니다.
서울에 딱 왔더니 임금이 없는 거에요.
일본의 무사들은요 아무리 작은 성의 성주라도 성을 버리고 도망가는 일이 없습니다
배가르고 죽거나 아니면 딱 항복해요.
그런데 세상에 성주도 아니고 나라의 임금 이라는 사람이 수도로 버리고 도망을 가니까. 
일본인들은 어 이거 어떡하지 우리 어떻게 해야 돼 그러면서 서울에서 추첨 평양 에서 주춤 주춤 시간을 보내고 있었던 거죠.
이 시간이 이순신 한텐  골든타임 이었습니다.
재 전열을 하고 일본을 칠 수 있는 시간을 벌어 준 겁니다.
아 거기서 일본의 2차 목표는 뭐냐면 자 육군은 1차로 치고 올라가죠.
그리고 2차 부대는 남해안을  돌아서 이렇게 서해안으로 상륙을합니다. 
일본 의 1차 부대에게 이제 무기와 식량을 공급하겠다.
이것이 그들의 계획이었는데 이건 뭐 이 이순신 장군께서 족족 끊어버렸어요.
해전에서 아예 그냥 씨를 말려 버립니다.
일본인들은 배고프잖아요 무기도 없잖아요 힘이 빠집니다.
그래서 1년만에 군대를 철수하게 되니 이것이 임진왜란이 됩니다.
'''

In [19]:
input_data = txt2input(text)
sum_list = test(args, input_data, -1, '', None)
sum_list[0]

[2021-11-01 10:33:12,626 INFO] Loading checkpoint from /content/gdrive/MyDrive/<융캡> 회의중 드라이브/문서요약/model_step_24000.pt의 사본
[2021-11-01 10:33:36,086 INFO] * number of parameters: 177854209


gpu_rank 0


array([[ 9, 16,  2, 14, 12, 11, 10, 17, 18,  3,  5, 13, 15,  1,  7,  4,
         8,  0,  6]])

In [20]:
[list(filter(None, text.split('\n')))[i] for i in sum_list[0][0][:3]]

['안되겠다 이들의 에너지를 표출할 수 있는 분출구를 찾아야겠다는 생각을 합니다.',
 '한양을 버리고 의주로 도망을 갑니다.',
 '여기는 일본을 주목 하셔야 되는데요.']